<a href="https://colab.research.google.com/github/Inteli-College/2024-2A-T05-CC09-G04/blob/detectron2/detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### O que é Detectron2?

**Detectron2** é uma biblioteca de visão computacional desenvolvida pelo Facebook AI Research (FAIR). Ela oferece uma plataforma eficiente e flexível para realizar tarefas avançadas de visão, como:

- **Detecção de Objetos**: Identificação de objetos em uma imagem por meio de bounding boxes (caixas delimitadoras).
- **Segmentação de Instâncias**: Delimitação precisa dos objetos, fornecendo máscaras detalhadas.
- **Segmentação Panóptica**: Combinação de detecção de objetos e segmentação semântica, permitindo classificar todos os pixels em uma imagem.

Detectron2 é amplamente utilizado em tarefas de **machine learning** que envolvem análise de imagens e oferece suporte a modelos de última geração, como **Faster R-CNN**, **Mask R-CNN**, **RetinaNet**, entre outros. Ele também oferece uma maneira fácil de treinar, testar e ajustar modelos pré-treinados com a capacidade de lidar com diferentes formatos de datasets, como o **COCO**.

Detectron2 se destaca por ser altamente otimizado para funcionar em GPUs, permitindo treinar e inferir modelos complexos de maneira eficiente.


In [ ]:
!python -m pip install pyyaml==5.1

!pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Outras bibliotecas úteis
!pip install opencv-python
!pip install matplotlib

In [ ]:
from PIL import Image
import os

# Caminho para o diretório de imagens
image_dir = '/content/drive/MyDrive/cattle_dataset'

# Definir o tamanho esperado
expected_size = (128, 128)

# Extensões de arquivo de imagem comuns
image_extensions = ['.png', '.jpg', '.jpeg']

# Loop por todas as imagens no diretório
for image_name in os.listdir(image_dir):
    image_path = os.path.join(image_dir, image_name)

    # Verifique se o arquivo é uma imagem com base na extensão
    if os.path.splitext(image_name)[1].lower() in image_extensions:
        try:
            # Abra a imagem
            with Image.open(image_path) as img:
                if img.size != expected_size:
                    print(f"Redimensionando {image_name} de {img.size} para {expected_size}")
                    img = img.resize(expected_size)
                    img.save(image_path)
        except Exception as e:
            print(f"Erro ao abrir a imagem {image_name}: {e}")
    else:
        print(f"Arquivo ignorado (não é uma imagem): {image_name}")

print("Redimensionamento completo!")


In [ ]:
import os
import json

# Caminho para os seus JSONs de train e test
train_json_path = '/content/drive/MyDrive/cattle_dataset/train_coco.json'
test_json_path = '/content/drive/MyDrive/cattle_dataset/test_coco.json'

# Função para atualizar as dimensões no JSON
def update_image_dimensions(json_path, correct_width, correct_height):
    # Carregar o JSON existente
    with open(json_path, 'r') as f:
        coco_data = json.load(f)

    # Atualizar as dimensões de cada imagem no JSON
    for image_info in coco_data['images']:
        # Corrigir as dimensões para todas as imagens
        if image_info['width'] != correct_width or image_info['height'] != correct_height:
            print(f"Corrigindo tamanho para {image_info['file_name']}: de ({image_info['width']}, {image_info['height']}) para ({correct_width}, {correct_height})")
            image_info['width'] = correct_width
            image_info['height'] = correct_height

    # Salvar o arquivo JSON atualizado
    updated_json_path = json_path.replace('.json', '_updated.json')
    with open(updated_json_path, 'w') as f:
        json.dump(coco_data, f)

    print(f"JSON atualizado salvo em {updated_json_path}")

# Definir as dimensões corretas (128x128)
correct_width = 128
correct_height = 128

# Atualizar os JSONs de treinamento e teste
update_image_dimensions(train_json_path, correct_width, correct_height)
update_image_dimensions(test_json_path, correct_width, correct_height)

print("Dimensões atualizadas para os arquivos JSON de train e test.")


In [ ]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import matplotlib.pyplot as plt


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

# Registrar os datasets de treino e teste
train_json = '/content/drive/MyDrive/cattle_dataset/train_coco.json'
test_json = '/content/drive/MyDrive/cattle_dataset/test_coco.json'
image_dir = '/content/drive/MyDrive/cattle_dataset'

# Registrar o dataset em formato COCO no Detectron2
register_coco_instances("cattle_train", {}, train_json, image_dir)
register_coco_instances("cattle_test", {}, test_json, image_dir)

# Verificar se o dataset foi registrado corretamente
metadata = MetadataCatalog.get("cattle_train")
dataset_dicts = DatasetCatalog.get("cattle_train")

print("Dataset registrado com sucesso!")


### Modelo Utilizado: Detection/faster_rcnn_R_50_FPN_3x.yaml

O modelo que estamos utilizando, **Detection/faster_rcnn_R_50_FPN_3x.yaml**, vem do **Model Zoo** do Detectron2 e refere-se a uma configuração específica de um modelo de detecção de objetos:

- **Faster R-CNN**: Este é um dos modelos mais populares para detecção de objetos. Ele usa uma abordagem de dois estágios:
  1. **Proposta de Região**: O modelo primeiro identifica regiões de interesse na imagem que podem conter objetos.
  2. **Classificação e Regressão de Bounding Box**: As regiões propostas são então refinadas, e o modelo prevê as classes dos objetos e ajusta as caixas delimitadoras.
  
- **R_50**: Refere-se ao backbone **ResNet-50**, que é uma arquitetura de rede neural convolucional usada para extrair características das imagens. ResNet-50 é uma versão mais leve e rápida comparada com backbones mais profundos (como ResNet-101), enquanto ainda mantém uma boa precisão.

- **FPN**: A **Feature Pyramid Network** (FPN) é uma técnica usada para melhorar a detecção de objetos em diferentes escalas. O FPN ajuda a detectar tanto objetos pequenos quanto grandes, o que é crucial em cenários onde os tamanhos dos objetos podem variar significativamente.

- **3x**: Indica que o modelo foi treinado por **3 vezes mais iterações** do que o padrão usado no dataset COCO. Isso significa que o modelo passou por mais etapas de ajuste dos seus pesos, melhorando a precisão e a robustez na detecção.

Esse modelo é eficiente para detecção de objetos e oferece uma boa precisão, equilibrando o uso de recursos computacionais e a capacidade de detectar objetos em diferentes escalas e contextos.

In [ ]:
import detectron2
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os
# Configuração do modelo
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cattle_train",)
cfg.DATASETS.TEST = ("cattle_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")  # Pesos pré-treinados no COCO
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0001  # Taxa de aprendizado
cfg.SOLVER.MAX_ITER = 1500    # Número de iterações de treinamento (ajuste conforme necessário)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32   # Este é o batch size padrão
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Temos apenas uma classe (face do gado)

# Salvar os pesos do modelo treinado
cfg.OUTPUT_DIR = "/content/drive/MyDrive/detectron2_output"

# Iniciar o treinamento
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
import cv2
from google.colab.patches import cv2_imshow  # Import para Colab

# Configuração do predictor para fazer inferências
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Pesos finais do modelo treinado
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # Definir um threshold
predictor = DefaultPredictor(cfg)

# Carregar uma nova imagem para testar
image_path = "/content/drive/MyDrive/framesfinal/boiteste.jpg"
im = cv2.imread(image_path)

# Fazer inferência
outputs = predictor(im)

# Visualizar o resultado
v = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=0.8)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])
